In [72]:
import numpy as np
import math
from typing import List

class Particle():
    def __init__(self, position, property):
        self.position = position
        self.complex_position = self.position[0] + 1J * self.position[1]
        self.property = property
        self.acceleration = np.array([0.,0.,0.])
        self.velocity = np.array([0.,0.,0.])

class Mesh():
    def __init__(self, width, n_levels, expansion_order):
        self.width = width
        self.n_levels = n_levels
        self.expansion_order = expansion_order
        # A 3D array indexed [level][i][j]
        self.meshboxes = []
        self.initialise_mesh()
    
    def initialise_mesh(self):
        # Index main levels from 0, ie level 0 is only one box
        for level in range(0, self.n_levels+1):
            self.meshboxes.append([])
            for i in range(0, 2**level):
                self.meshboxes[level].append([])
                for j in range(0, 2**level):
                    self.add_meshbox(level, [i,j])
            for i in range(0, 2**level):
                for j in range(0, 2**level):
                    self.meshboxes[level][i][j].allocate_neighbours()
            for i in range(0, 2**level):
                for j in range(0, 2**level):
                    self.meshboxes[level][i][j].calc_i_list()
            


    def add_meshbox(self, level, level_coords):
        if level-1 > -1:
            new_meshbox_parent = self.meshboxes[level-1][int(level_coords[0]/2)][int(level_coords[1]/2)]
            new_meshbox = MeshBox(self, new_meshbox_parent, level, level_coords)
            new_meshbox_parent.children.append(new_meshbox)
        else:
            new_meshbox = MeshBox(self, None, level, level_coords)
        
        self.meshboxes[level][level_coords[0]].append(new_meshbox)
    
    def add_particle(self, particle: Particle):
        fine_level_coords = np.int64(np.floor(particle.position/self.width * 2**self.n_levels))
        self.meshboxes[self.n_levels][fine_level_coords[0]][fine_level_coords[1]].add_particle(particle)
    
    def calc_fine_mpes(self):
        for meshbox in list(np.concatenate(self.meshboxes[self.n_levels]).flat):
            meshbox.calc_fine_mpe()
    
    def calc_coarse_mpes(self):
        for level in reversed(range(0, self.n_levels)):
            for meshbox in list(np.concatenate(self.meshboxes[level]).flat):
                meshbox.calc_coarse_mpe()
    
    def calc_local_expansions(self):
        for level in range(1, self.n_levels+1): # From 1...n-1
            for meshbox in list(np.concatenate(self.meshboxes[level]).flat):
                meshbox.calc_local_expansion()
            if level < self.n_levels +1:
                for meshbox in list(np.concatenate(self.meshboxes[level]).flat):
                    meshbox.translate_le_to_children()
    
    def calc_le_particle_potentials(self):
        for meshbox in list(np.concatenate(self.meshboxes[self.n_levels]).flat):
            meshbox.evaluate_particle_les()
    
    def calc_neighbour_particle_potentials(self):
        for meshbox in list(np.concatenate(self.meshboxes[self.n_levels]).flat):
            meshbox.evaluate_neighbour_potentials()
            

class MeshBox():
    def __init__(self, mesh: Mesh, parent, level, level_coords):
        self.mesh = mesh
        self.level = level
        self.level_coords = level_coords
        self.width = mesh.width / (2 ** level)
        self.children = []
        self.parent = parent
        self.particles = []
        self.centre = (np.array(level_coords) + np.array([0.5, 0.5])) / 2**level * mesh.width
        self.complex_centre = self.centre[0] + 1j*self.centre[1]
        self.i_list : List[MeshBox] = []
        self.neighbours = []
    
    def allocate_neighbours(self):
        neighbour_list = []
        for i in range(-1,2):
            for j in range(-1,2):
                if self.level_coords[0]+i > -1 and self.level_coords[1]+j > -1 and self.level_coords[0]+i < 2**self.level and self.level_coords[1]+j < 2**self.level:
                    neighbour_list.append(self.mesh.meshboxes[self.level][self.level_coords[0]+i][self.level_coords[1]+j])
        neighbour_list.remove(self)
        self.neighbours = neighbour_list
    
    def calc_i_list(self):
        i_list = [] # Interaction list
        if self.parent != None:
            for parent_neighbour in self.parent.neighbours:
                i_list += parent_neighbour.children
            for own_neighbour in self.neighbours:
                if own_neighbour in i_list:
                    i_list.remove(own_neighbour)
            self.i_list = i_list
    
    def add_particle(self, particle: Particle):
        self.particles.append(particle)
        if self.parent != None:
            self.parent.add_particle(particle)
    
    def calculate_total_property(self):
        self.property_total = sum([particle.property for particle in self.particles])
        return self.property_total
    
    def calc_fine_mpe(self):
        coefficients = [self.calculate_total_property()]
        if self.mesh.expansion_order < 0:
            raise Exception("For multipole expansion, p must be greater than 0")
        for exponent in range(1, self.mesh.expansion_order+1):
            coefficient = 0
            for particle in self.particles:
                coefficient += (-particle.property * (particle.complex_position - self.complex_centre) ** exponent) / exponent
                #print((particle.complex_position - self.complex_centre))
                #print(f'particle {particle.complex_position}')
                #print(f'box {self.complex_centre}')
            coefficients.append(coefficient)
        # Includes a_0 (Q) up to a_n
        self.mpe_coefficients = coefficients
    
    def calc_coarse_mpe(self):
        shift_coeffs = np.zeros(self.mesh.expansion_order+1, dtype='complex128')
        for child_meshbox in self.children:
            child_coeffs = child_meshbox.mpe_coefficients
            # Initialise the array and include the (unchanged) a_0
            child_shift_coeffs = [child_coeffs[0]]
            # Calculates the remaining b_l
            for shift_exponent in range(1, self.mesh.expansion_order + 1):
                child_shift_coeff = 0
                for k in range(1, shift_exponent+1):
                    z_0 = child_meshbox.complex_centre - self.complex_centre
                    child_shift_coeff += child_coeffs[k] * (z_0) ** (shift_exponent-k) * math.comb(shift_exponent-1, k-1)
                child_shift_coeff += (-child_coeffs[0] * z_0 ** shift_exponent) / shift_exponent
                child_shift_coeffs.append(child_shift_coeff)
                
            #print(len(shift_coeffs))
            #print(len(child_shift_coeffs))
            shift_coeffs += child_shift_coeffs
        self.mpe_coefficients = shift_coeffs
        #print(shift_coeffs)

    def calc_local_expansion(self):
        if self.level == 1:
            le_coeffs = np.zeros(self.mesh.expansion_order+1, dtype='complex128') # Local expansion coefficients
        else:
            le_coeffs = self.le_coeffs

        for i_box in self.i_list:
            z_0 = i_box.complex_centre - self.complex_centre
            # Calculating b_0
            b_0 = np.complex128(0)
            for k in range(1, self.mesh.expansion_order+1):
                b_0 += i_box.mpe_coefficients[k]/(z_0**k) * (-1)**k + i_box.mpe_coefficients[0] * np.log(-z_0)
            le_coeffs[0] += b_0
            for l in range(1, self.mesh.expansion_order+1):
                b_l = np.complex128(0)
                for k in range(1,self.mesh.expansion_order+1):
                    #print(l, k)
                    b_l += 1/(z_0**l) * i_box.mpe_coefficients[k] / (z_0**k) * math.comb(l+k-1, k-1) * (-1)**k
                b_l -= i_box.mpe_coefficients[0]/(l*(z_0**l))
                le_coeffs[l] += b_l
        self.le_coeffs = le_coeffs
        #print(self.level)
        #print(le_coeffs)
    
    def translate_le_to_children(self):
        for child_box in self.children:
            z_0 = child_box.complex_centre - self.complex_centre
            child_le_coeffs = np.zeros(self.mesh.expansion_order+1, dtype='complex128')
            for l in range(0, self.mesh.expansion_order+1):
                for k in range(1, self.mesh.expansion_order+1):
                    child_le_coeffs[l] += self.le_coeffs[k] * math.comb(k,l) * (-z_0)**(k-l)
            child_box.le_coeffs = child_le_coeffs

    def evaluate_particle_les(self):
        for particle in self.particles:
            particle.le_potential = self.evaluate_le(particle)
    
    def evaluate_le(self, particle: Particle):
        le_potential = np.complex128(0)
        z = particle.complex_position - self.complex_centre
        for l in range(0, self.mesh.expansion_order+1):
            le_potential += self.le_coeffs[l] * z**l
        return le_potential
    
    def evaluate_neighbour_potentials(self):
        box_particles = self.particles
        neighbour_particles = []
        for neighbour in self.neighbours:
            neighbour_particles += neighbour.particles
        for box_particle in box_particles:
            particle_neighbour_potential = np.complex128(0)
            for neighbour_particle in neighbour_particles:
                


        
            
            

            

    
    

# Gives distance from point 1 to point 2
def distance(point_1, point_2):
    return np.linalg.norm(point_2-point_1)
        



In [73]:
n_particles = 1000
precision = 40
box_size = 1000
# property could be mass or charge 
max_property = 10
n_levels = int(np.ceil(np.emath.logn(4, n_particles)))
p = int(np.ceil(np.log2(precision)))
print(f'n_levels {n_levels}')
print(f'p {p}')
initial_positions = np.random.random((n_particles,2)) * box_size
initial_particles = []
for initial_position in initial_positions:
    initial_particles.append(Particle(initial_position, np.random.uniform(0,max_property)))

# Initialisaiton
mesh = Mesh(box_size, n_levels, p)
for particle in initial_particles:
    mesh.add_particle(particle)

# FMM
mesh.calc_fine_mpes() # Step 1
mesh.calc_coarse_mpes() # Step 2
mesh.calc_local_expansions() # Step 3 and 4
mesh.calc_le_particle_potentials() # Step 5
mesh.calc_neighbour_particle_potentials() # Step 6






n_levels 5
p 6


In [74]:
# print(mesh.meshboxes[1][0][1].level_coords)
# print(mesh.meshboxes[4][3][3].parent.parent.parent.parent.parent)
# print(len(mesh.meshboxes))
# for meshbox in mesh.meshboxes:
#     print(len(meshbox))
# print(len(mesh.meshboxes[7][1][3].particles))
# for i in range(0,50):
#     print(mesh.meshboxes[7][5][i].mpe_coefficients)
#     print(mesh.meshboxes[7][5][i].calculate_total_property())
# print(mesh.meshboxes[0][0][0].particles[0].position)
for meshbox in mesh.meshboxes[5][15][8].i_list:
    if meshbox.particles != []:
        print(meshbox.particles[0].le_potential)
    #print(meshbox.particle)


(4405.3207845618845-492.1523329308778j)
(4746.309137323549-209.25300074515616j)
(4520.743647549014-98.45599713965127j)
(2843.9186610800407-734.6021563251213j)
(2312.2986788333574+541.8818207962106j)
(2893.7761516286514+290.72532917081884j)
(4749.159981718017-589.179943289601j)
(3644.7566425493374+377.9201904661129j)
(3237.7971868558325+380.8853428125209j)
(3863.588151512563+710.8823012489096j)
(3875.2653927936703-390.143999385861j)
(4223.93852531911-443.64554156957735j)
(3172.677977848819-336.7290854938755j)
(3041.2517766784385-169.24968219945674j)
(3910.736305231122-371.47637962100856j)
(4179.624381006854-217.48153352068655j)
(3881.3478624463396-40.71234120869627j)
